In [1]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
sns.set(rc={'figure.figsize':(16,10)})

In [3]:
year = '19'

In [4]:
f = uproot.pandas.iterate("../outputs/"+year+"/*.root", "t")
data = pd.concat([j for j in f])[['label', 'be', 'm', 'proc', 'chi', 'm_rec']]

AttributeError: can't set attribute

In [ ]:
lum = pd.read_csv('../inputs/'+year+'/lum', header=None, index_col=[0])
lum.columns = ['lum']

In [ ]:
lum.head()

Sort values by label

In [ ]:
data.sort_values(by='label', ascending=True, inplace=True);
data.shape

Cut off incorrect beam energy

In [ ]:
data.query('be>0&chi<3', inplace=True)
data.shape

Change index column

In [ ]:
data.index = data.label
data.drop(['label'], axis=1, inplace=True)

Join lum to data table

In [ ]:
data = data.join(lum, how='inner')

In [ ]:
#cnct = [] #model
cnct = [ ([900.0, 912.5, 925.0, 935.0, 945., 950.0, 962.5, 975.0, 987.5, 1000.0], 950), 
       ([825.0, 837.5, 850.0, 862.5, 875.0, 887.5], 850), ([800.0, 812.5],810), ([775.0, 787.5], 780),
       ([725.0, 737.5, 750.0, 762.5], 740), ([662.5, 675.0, 687.5, 700.0, 712.5], 680),
       ([625.0, 637.5, 650.0], 637)] #11year

Create dat dictionary with three procedures

In [ ]:
data['label'] = data.index
for c in cnct:
    data.label.replace(c[0], c[1], inplace=True)
data.index = data.label
data.drop(['label'], axis=1, inplace=True)

In [ ]:
data.query('proc==3&abs(m_rec-497)<50').groupby('label').agg({'m_rec': np.size})

In [ ]:
energy = data.assign(elum=data['be']*data['lum'])[['be', 'lum', 'elum']].drop_duplicates() \
        .groupby('label').agg({'elum': np.mean, 'lum': [np.mean, np.sum], 'be':[np.min, np.max]})
energy['emean'] = energy[('elum','mean')]/energy[('lum','mean')]
energy['err_l'] = energy['emean'] - energy[('be', 'amin')]
energy['err_r'] = energy[('be', 'amax')] - energy['emean']
energy['L'] = energy[('lum', 'sum')]
energy = energy[['emean', 'err_l', 'err_r', 'L']]

## Table with energy points information

In [ ]:
energy

## Get table for fit using its rows

In [ ]:
mass_bound = (450, 550)

In [ ]:
dt = pd.DataFrame( {'type':'kinfit', 
        'mass':data.query('proc!=2&m_rec>@mass_bound[0]&m_rec<@mass_bound[1]')[['m_rec']].groupby('label').
                    apply(np.array).apply(np.ravel)}) 

In [ ]:
dt = dt.append( pd.DataFrame({'type':'standard', 
                    'mass':data.query('proc!=1&m>@mass_bound[0]&m<@mass_bound[1]')[['m']].groupby('label').
                              apply(np.array).apply(np.ravel)}))

In [ ]:
dt = dt.append( pd.DataFrame( {'type':'both', 
        'mass':data.query('proc==3&m_rec>@mass_bound[0]&m_rec<@mass_bound[1]')[['m_rec']].groupby('label').
                               apply(np.array).apply(np.ravel)}) )

In [ ]:
dt['label'] =dt.index
dt.index = range(len(dt))

In [ ]:
dt = dt.sort_values(by=['label', 'type'])

In [ ]:
dt.head()

## FIT

In [ ]:
import iminuit
import probfit as pf

In [ ]:
def constant(x):
    return 1
norm_bkg = pf.Normalized( constant, mass_bound )
norm_bkg = pf.Extended(norm_bkg, extname='Nbkg')

gauss1 = pf.Extended(pf.rename(pf.gaussian, ['x', 'Mass', 'Sigma']), extname='Nsig')

pdf = pf.AddPdf(gauss1, norm_bkg)
pf.describe(pdf)

In [ ]:
def fitter(dic, nbins):
    likelihood = pf.BinnedLH(pdf, dic, nbins, bound=mass_bound, extended=True)
    initial_par_values = {"Mass":497.6, "Sigma":5.6, "Nsig":np.size(dic), "Nbkg":0}
    errors = {"error_Mass": 1, "error_Sigma": 0.1, "error_Nsig": 0.3, "error_Nbkg":0}
    limits = {"limit_Mass": [450, 550], "limit_Sigma": [2, 15], "limit_Nsig": [1, 1000], "limit_Nbkg": [0, 1000]}
    mt = iminuit.Minuit(likelihood, **initial_par_values, **errors, **limits, pedantic=False)
    mt.migrad();
    likelihood.draw(minuit=mt)
    return mt

In [ ]:
def fitter_gauss(dic, nbins):    
    likelihood = pf.BinnedLH(gauss1, dic, nbins, bound=mass_bound, extended=True)
    initial_par_values = {"Mass":497.6, "Sigma":6, "Nsig":np.size(dic)}
    errors = {"error_Mass": 0.1, "error_Sigma": 0.1, "error_Nsig": 0.3}
    mt = iminuit.Minuit(likelihood, **initial_par_values, **errors)
    mt.migrad();
    likelihood.draw(minuit=mt)
    return mt

In [ ]:
itr = iter(dt.iterrows())

Fitting procedure

In [ ]:
i,j = next(itr)
print('Index:', i, '| Label:', j['label'], '| Type:', j['type'])

In [ ]:
mass_bound = (450, 550)
mt = fitter(j['mass'], 50)
dt.at[i, 'Nsig'] = mt.values[2]
dt.at[i, 'ErrS'] = mt.errors[2]
print('Accurate matrix:', mt.matrix_accurate())
print("Migrad quality:", mt.migrad_ok())
print( 'Real:', np.size(j['mass']), 'Found:', mt.values[2] + mt.values[3] )

In [ ]:
ind = 2
dt.loc[ind:]

In [ ]:
table_numbersM = pd.pivot_table(dt, values=['Nsig'], index=['label'], columns=['type'])

In [ ]:
table_numbersM['rec'] = table_numbersM[('Nsig','both')]/table_numbersM[('Nsig','kinfit')]
table_numbersM['err_rec'] = table_numbersM.rec*np.sqrt(1/table_numbersM.Nsig.both + 1/table_numbersM.Nsig.kinfit)

In [ ]:
table_numbersM = table_numbersM.join(energy, how='inner')

In [ ]:
table_numbersM

In [ ]:
table_numbersM.to_csv('regEffModel_new.csv')

Comparison with modelling

In [ ]:
table_numbers = pd.read_csv('regEffModel.csv', index_col=['Unnamed: 0'])

In [ ]:
table_numbers = pd.pivot_table(table_numbers, values=['Nsig'], index=['label'], columns=['type'])

In [ ]:
table_numbers['effS'] = table_numbers[('Nsig','both')]/table_numbers[('Nsig','kinfit')]

## Working with saved files

In [ ]:
def prep_t(t):
    table = pd.pivot_table(t, columns=['type'], index=['label'], values=['ErrS','Nsig'])
    table.columns = ['err_b', 'err_k', 'err_s', 'n_b', 'n_k', 'n_s']
    table['err_b'] = table['err_b']/table['n_b'] 
    table['err_k'] = table['err_k']/table['n_k']
    table.drop(['err_s', 'n_s'], axis=1, inplace=True)
    table['rec'] = table['n_b']/table['n_k']
    table['err_rec'] = table['rec']* np.sqrt( np.square(table['err_b']) + np.square(table['err_k']) )
    table.drop(['err_b', 'err_k', 'n_b', 'n_k'], axis=1, inplace=True)
    return table

In [ ]:
def prep_t3(t):
    t.columns = ['Nb', 'Nk', 'rec', 'err_rec_old', 'E', 'err_left', 'err_right', 'L']
    t['rec'] = (t.Nb+1)/(t.Nk+2)
    t['err_rec'] = np.sqrt( (t.Nb+1)*(t.Nb+2)/(t.Nk+2)/(t.Nk+3) - np.square((t.Nb+1)/(t.Nk+2)) )
    #t.drop([ 'Nb', 'Nk'], axis=1, inplace=True)
    return

In [ ]:
def prep_t2(t):
    t.columns = ['n_b', 'n_k', 'n_s', 'rec']
    t.drop(['n_s'], axis=1, inplace=True)
    t['err_b'] = np.sqrt( 1./t['n_b'] )
    t['err_k'] = np.sqrt( 1./t['n_k'] )
    t['err_rec'] = t['rec']* np.sqrt( np.square(t['err_b']) + np.square(t['err_k']) )
    t.drop(['err_b', 'err_k', 'n_b', 'n_k'], axis=1, inplace=True)
    return

In [ ]:
t11 = pd.read_csv('regEff11_new.csv', index_col=[0], header=[0,1])
t12 = pd.read_csv('regEff12.csv', index_col=[0], header=[0,1])
t17 = pd.read_csv('regEff17.csv', index_col=[0], header=[0,1]).dropna()
tM  = pd.read_csv('regEffModel_new.csv', index_col=[0], header=[0,1])

Model: additional merge

In [ ]:
prep_t3(t11)
prep_t3(tM)

In [ ]:
prep_t2(t12)
prep_t2(t17)

In [ ]:
tM

In [ ]:
cnct = [ ([900.0, 912.5, 925.0, 935.0, 945., 950.0, 962.5, 975.0, 987.5, 1000.0], 950), 
       ([825.0, 837.5, 850.0, 862.5, 875.0, 887.5], 850), ([800.0, 812.5],810), ([775.0, 787.5], 780),
       ([725.0, 737.5, 750.0, 762.5], 740), ([662.5, 675.0, 687.5, 700.0, 712.5], 680),
       ([625.0, 637.5, 650.0], 637)] #11year

In [ ]:
tM['label'] = tM.index
for c in cnct:
    tM.label.replace(c[0], c[1], inplace=True)
tM.index = tM.label
tM.drop(['label'], axis=1, inplace=True)

In [ ]:
tM = tM.groupby('label').agg({'Nb':np.sum, 'Nk':np.sum, 'rec':np.mean, 'err_rec_old':np.mean, 'E':np.mean, 
                              'err_left':np.mean, 'err_right':np.mean, 'L':np.sum})
prep_t3(tM) #wrong energy errors, but they don't necessary for the analysis

### Division

In [ ]:
div_table = t11.copy()

In [ ]:
div_table.head()

In [ ]:
div_table = div_table.join(tM[['rec', 'err_rec']].rename(columns={'rec':'recM', 'err_rec':'err_recM'}), how='inner')

In [ ]:
div_table = div_table[['rec','err_rec', 'recM', 'err_recM']]

In [ ]:
div_table['corr'] = div_table.rec/div_table.recM
div_table['err_corr'] = div_table['corr']*np.sqrt( np.square(div_table.err_rec/div_table.rec) + \
                                               np.square(div_table.err_recM/div_table.recM) )

In [ ]:
div_table

### Data have prepared. It's time for drawing

**2011 vs model**

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.errorbar(x=t11.index, y=t11.rec, xerr=t11.err_left, yerr=t11.err_rec,  fmt='o', color='lightgreen', elinewidth=3, label='2011')
plt.errorbar(x=tM.index, y=tM.rec, yerr=tM.err_rec,  fmt='*', color='black', elinewidth=3, label='model')
axes = plt.gca()
axes.set_ylim([0,1]);
axes.legend(fontsize=20)

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.errorbar(x=div_table.index, y=div_table['corr'], yerr=div_table.err_corr,  fmt='*', color='blue', elinewidth=3, label='2011')
axes = plt.gca()
axes.set_ylim([0.5,1.9]);
axes.legend(fontsize=20)

**2017 vs model**

In [ ]:
plt.errorbar(x=t17.index, y=t17.rec, yerr=t17.err_rec,  fmt='o', color='lightgreen', elinewidth=3, label='2017')
plt.errorbar(x=tM.index, y=tM.rec, yerr=tM.err_rec,  fmt='*', color='black', elinewidth=3, label='model')
axes = plt.gca()
axes.set_ylim([0,1.5]);
axes.legend(fontsize=20)

**Rec eff 11**

In [ ]:

plt.errorbar(x=t11.index, y=t17.rec, yerr=t17.err_rec,  fmt='o', color='lightgreen', elinewidth=3, label='2017')
axes = plt.gca()
axes.set_ylim([0,1.5]);
axes.legend(fontsize=20)